# Evaluation <a name = "Top"></a>

# Quick Links

<ol>
    <li><a href = #setup>Setup</a></li>
    <li><a href = #plots>Plots</a></li>
</ol>

## Imports

In [ ]:
import os
import shutil

In [ ]:
from datetime import datetime, timedelta
import time
import pandas as pd
import numpy as np
import shutil

from enum import Enum

from numpy import array

import ntpath

import copy
import re

## Hilfsfunktionen

### Enum für Trainingsset

In [ ]:
class TrainingSet(Enum):
    SYNTHETIC = 1
    REAL = 2
    MIXED = 3

### Enum für Label-Typ

In [ ]:
class LabelType(Enum):
    ANGULAR = 1
    STEREOGRAPHIC = 2

### Trainingsset-Typ nach String Converter

In [ ]:
def trainingset_to_string(ts):
    if ts == TrainingSet.SYNTHETIC:
        return 'Synth'
    elif ts == TrainingSet.REAL:
        return 'Real'
    elif ts == TrainingSet.MIXED:
        return 'Mixed'
    else:
        print('Unknown TrainingSet')
        return None

### LabelType nach String

In [ ]:
def labeltype_to_string(lt):
    if lt == LabelType.ANGULAR:
        return 'Angular'
    if lt == LabelType.STEREOGRAPHIC:
        return 'Stereographic'
    else:
        print('Unknown LabelType')
        return None

#### Required format of parameters parameter for _model_predict_ (...)

In [ ]:
p = {
    'dataset_to_use':[],
    'model_to_load':[],
    'dataset_name':'combined_dataset',
    'activation_function':[],
    'leaky_ReLU_alpha':[],
    'first_neuron':[],
    'dropout_rate':[],
    'hidden_layers':[],
    'optimizer':[],
    'learning_rate':[],
    'loss_function':[],
    'label_type':[]
}

## Hilfsfunktionen

### Konvertierung ($S_x$, $S_y$) $\rightarrow$ ($\phi$, $\theta$)

In [ ]:
def convert_from_stereographic(sx, sy, r = 1):
    
    l = np.sqrt(sx * sx + sy * sy)
    theta = 90 - 2 * np.degrees(np.arctan(l / 2))
    
    if sx < 0:
        phi = 180 - np.degrees(np.arcsin(sy / l))
        
    elif sx >= 0:
        if sy > 0:
            phi = np.degrees(np.arcsin(sy / l))
        elif sy < 0:
            phi = 360 + np.degrees(np.arcsin(sy / l))
        else:
            #phi1 = np.NaN
            phi = 0
    else:
        print('sx and sy undefined. should not have reached here')

    return phi, theta

### Konvertierung ($\phi$, $\theta$) $\rightarrow$ ($S_x$, $S_y$)

In [ ]:
def convert_from_spheric(phi, theta, r = 1):
    m = 2 * r * np.tan(np.radians((90 - theta) / 2))
    sy = m * np.sin(np.radians(phi))
    sx = m * np.cos(np.radians(phi))
    return sx, sy

### Radians $\rightarrow$ Degree

In [ ]:
def to_degree(angle_in_rad):
    return angle_in_rad * 180 / np.pi

### Degree $\rightarrow$ Radians

In [ ]:
def to_radians(angle_in_deg):
    return angle_in_deg * np.pi / 180

### Sphärische $\rightarrow$ Karthesische Koordinaten

In [ ]:
def spheric_cartesian_polar(phi_d, theta_d):
    x = np.sin(np.radians(90.0 - theta_d)) * np.cos(np.radians(phi_d))
    y = np.sin(np.radians(90.0 - theta_d)) * np.sin(np.radians(phi_d))
    z = np.cos(np.radians(90.0 - theta_d))
    return array([x, y, z])

In [ ]:
def spheric_cartesian_elevation(phi_d, theta_d):
    x = np.cos(np.radians(theta_d)) * np.cos(np.radians(phi_d))
    y = np.cos(np.radians(theta_d)) * np.sin(np.radians(phi_d))
    z = np.sin(np.radians(theta_d))
    return array([x, y, z])

### Length of Vector

In [ ]:
def vectorlength(vector):
    return np.linalg.norm(vector)

### Calculated Angular Error

In [ ]:
def calculate_angular_error(deg_e_phi, deg_e_theta):
    return np.degrees(np.arccos(np.cos(np.radians(deg_e_phi)) * np.cos(np.radians(deg_e_theta))))

### Skalarprodukt

In [ ]:
def myDot(a, b):
    dot = 0;
    it = np.nditer(a, flags=['f_index'])
    for x in it:
        dot = dot + (x * b[it.index])
        
    return dot

### Dot Angular Error

In [ ]:
def dot_angular_error_elevation(predicted_deg_vector, true_deg_vector):    
    c_predicted = spheric_cartesian_elevation(predicted_deg_vector[0], predicted_deg_vector[1])
    c_true = spheric_cartesian_elevation(true_deg_vector[0], true_deg_vector[1])
    
    len_prediction = vectorlength(c_predicted)
    len_true = vectorlength(c_true)
    
    cos_angle = np.dot(c_true, c_predicted) / len_prediction / len_true
    
    return abs(np.degrees(np.arccos(cos_angle)))

def dot_angular_error_polar(predicted_deg_vector, true_deg_vector):
    c_predicted = spheric_cartesian_polar(predicted_deg_vector[0], predicted_deg_vector[1])
    c_true = spheric_cartesian_polar(true_deg_vector[0], true_deg_vector[1])
    
    len_prediction = vectorlength(c_predicted)
    len_true = vectorlength(c_true)
    
    cos_angle = np.dot(c_true, c_predicted) / len_prediction / len_true
    
    return abs(np.degrees(np.arccos(cos_angle)))

## Normierte sphärische Koordinaten

### Normierte $\rightarrow$ Sphärische

In [ ]:
def normalized_to_spheric(n_phi, n_theta):
    phi = n_phi * 180 + 180
    theta = n_theta * 45 + 45
    
    return phi, theta

# Evaluation <a name = "setup"></a>
<p><a href = #Top>Up</a>
<p><a href = #plots>Plots</a>

In [ ]:
run = 'SYNTH'
loss = 'MSE'
dataset_name = '2020-05-28'
net_index = [99, 204, 195]
APPENDIX = ['Angular', 'Normalized', 'Stereographic']
_note = ['', '', '_Custom-MAE']

eval_dir = '..\\output\\{}_Regression_{}\\Graphical_Evaluation\\'.format(run, loss)

if(not os.path.exists(eval_dir)):
    os.makedirs(eval_dir)
else:
    input('Directory >>| {} |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)'.format(eval_dir))

In [ ]:
import pickle

net_results = [None, None, None]

for i in range(3):
    with open(eval_dir + '{}_Net{}_{}{}_Results.pickle'.format(dataset_name, net_index[i], APPENDIX[i], _note[i]), "rb") as fp:   # Unpickling
        net_results[i] = pickle.load(fp)

In [ ]:
for i in range(3):
    print(net_results[i])
    print()
    print()
    print('---------------------------------------------------------------------')
    print()
    print()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.text as text
from matplotlib.pyplot import figure

from matplotlib import cm

import tikzplotlib
%matplotlib inline

Enable_Plotting = True

# Plots <a name = "plots">
<p></a><a href = #Top>Up</a>
<p><a href = #setup>Setup</a>

In [ ]:
if Enable_Plotting:
    #interesting_cases = [0, 3, 4, 5]
    interesting_cases = [0, 5]
    interesting_cases = [0, 5]
    
    net_len = len(net_results)
    net_len = 3
    
    net_df = [[], [], []]
    
    for net_idx in range(net_len):
        net = net_results[net_idx]
        for run_idx in interesting_cases:
            df = net[run_idx][1]
            net_df[net_idx].append(df)
            #print(df)
    print(net_df[2][0])

In [ ]:
print(net_df[0][0])

##  Elevation-Dependent Angular Error

In [ ]:
def model_dependent_angular_error(dataframe, eval_path, eval_file, store_to_file = False):
    model_depenent_angular_error =  pd.DataFrame(data=None, index=None, columns=['model', 'angular_error', 'number_images'])
    
    operation_df = dataframe
    
    dictionary = {}
    
    for index, row in operation_df.iterrows():
        filename = row['Filename']
        if 'bunny' in filename:
            key = 'Bunny'
        elif 'buddha' in filename:
            key = 'Buddha'
        elif 'box' in filename:
            key = 'Box'
        elif 'cone' in filename:
            key = 'Cone'
        elif 'sphere' in filename:
            key = 'Sphere'
        elif 'ball' in filename:
            key = 'Sphere'
        else:
            print('No proper String: ', filename)
            return
        
        if key not in dictionary.keys():
            e = row['dot_angular_err_elevation']
            a = 1
            dictionary[key] = [e, a]
            
        else:
            e = dictionary[key][0]
            a = dictionary[key][1]
            e = e + row['dot_angular_err_elevation']
            a = a + 1
            dictionary[key] = [e, a]
            
    for key, value in dictionary.items():
        avg_error = value[0] / value[1]
        model_depenent_angular_error = model_depenent_angular_error.append(pd.DataFrame(
            data = [[key, avg_error, value[1]]], 
            index = None, 
            columns = ['model', 'angular_error', 'number_images']))

    model_depenent_angular_error = model_depenent_angular_error.sort_values(by=['model'])

    if(store_to_file):
        model_depenent_angular_error.to_csv(eval_path + eval_file, index = False)
    
    return model_depenent_angular_error

In [ ]:
model_dependent_angular_error(net_df[0][0], eval_dir, 'model_angular_99_train-synth_test-synth.csv', store_to_file = True)
model_dependent_angular_error(net_df[1][0], eval_dir, 'model_angular_204_train-synth_test-synth.csv', store_to_file = True)
model_dependent_angular_error(net_df[2][0], eval_dir, 'model_angular_195_train-synth_test-synth.csv', store_to_file = True)

model_dependent_angular_error(net_df[0][1], eval_dir, 'model_angular_99_train-mixed_test-real.csv', store_to_file = True)
model_dependent_angular_error(net_df[1][1], eval_dir, 'model_angular_204_train-mixed_test-real.csv', store_to_file = True)
model_dependent_angular_error(net_df[2][1], eval_dir, 'model_angular_195_train-mixed_test-real.csv', store_to_file = True)

In [ ]:
def do_Plot(eval_file, save = True):
    #fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (30,10))
    fig, ax = plt.subplots(figsize = (10,5))
    
    synt1 = model_dependent_angular_error(net_df[0][0], None, None) # Net 99 Tr syn Te syn
    synt2 = model_dependent_angular_error(net_df[1][0], None, None) # Net 204 Tr syn Te syn
    synt3 = model_dependent_angular_error(net_df[2][0], None, None) # Net 195 Tr syn Te syn
    
    real1 = model_dependent_angular_error(net_df[0][1], None, None) # Net 99 Tr syn Te syn
    real2 = model_dependent_angular_error(net_df[1][1], None, None) # Net 204 Tr syn Te syn
    real3 = model_dependent_angular_error(net_df[2][1], None, None) # Net 195 Tr syn Te syn

    
    synt_dfs = [synt1[['model', 'angular_error']], synt2['angular_error'], synt3['angular_error']]
    real_dfs = [real1[['model', 'angular_error']], real2['angular_error'], real3['angular_error']]
    
    synt_plot = pd.concat(synt_dfs, join = 'outer', axis = 1)
    synt_plot.reset_index(drop = True, inplace = True)
    
    real_plot = pd.concat(real_dfs, join = 'outer', axis = 1)
    real_plot.reset_index(drop = True, inplace = True)
    
        
    #synt_plot.plot(x = 'model', kind = 'bar', ax = ax)
    real_plot.plot(x = 'model', kind = 'bar', ax = ax)


    ax.grid(True, ls = '--', axis = 'y')
    
    ax.set_ylabel('Mean Angular Error')
    #ax.set_title('Model-dependent angular error on synth. test data using synth. training data')
    ax.set_title('Model-dependent angular error on real test data using mixed training data')
    
    ax.set_xlabel('Model')
    
    ax.legend(['Net_p,t', 'Net_|p,t|', 'Net_sx,sy'])
    
    for o in fig.findobj(text.Text):
        o.set_fontstyle('italic')
    
    if(save):
        tikzplotlib.save('{}.tex'.format(eval_file))
        plt.savefig('{}.png'.format(eval_file), format = 'png', bbox_inches = "tight", dpi = 300)

    plt.show()
    return

### Show Plot

In [ ]:
if Enable_Plotting:
    tsts_plot = do_Plot(eval_dir + 'Model-Dep-Angular')